 # TD 2: Focus on interpretability
 In this practical class, we will implement a method to investigate the importance of the features inputed to
 the classifier, in order to interpret how the classifier has learnt to distinguish healthy controls from depression
 patients using voice only.
 In a first approach, we will only focus on the eGeMAPS features.


## 1 Linear models (*)
In this section (and in this section only), we will use a SVM with a linear kernel.
### 1.1 Read speech
**Task n°1. Implement and evaluate this classifier on the read subcorpus of the Android corpus.**
Since the classifier is linear, it is possible to extract the coefficients of the classifier for each feature.\
**Task n°2. Extract the coefficient of the linear SVM for each fold.**

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC

# --- 1. FONCTION DE CHARGEMENT DES DONNÉES ---
def load_and_preprocess_data(base_path="/Users/klockjoely/Downloads/feautres_android_means/reading_means"):
    """Charge et prépare les données HC et PT du sous-corpus 'reading means'."""
    data_list = []
    labels_list = []
    
    # 1. Charger les données HC (Sains) -> Étiquette 0
    hc_path = os.path.join(base_path, 'HC')
    # Utilisez try-except pour gérer les erreurs si le chemin n'est pas trouvé
    try:
        for filename in os.listdir(hc_path):
            if filename.endswith('.csv'):
                filepath = os.path.join(hc_path, filename)
                df = pd.read_csv(filepath)
                data_list.append(df)
                labels_list.append(0) # Étiquette 0 pour HC
    except FileNotFoundError:
        print(f"Erreur: Le chemin HC n'a pas été trouvé: {hc_path}")
        return np.array([]), np.array([]), [] # Retourne des tableaux vides en cas d'erreur
    except Exception as e:
        print(f"Une erreur s'est produite lors du chargement des données HC : {e}")
        return np.array([]), np.array([]), []

    # 2. Charger les données PT (Malades) -> Étiquette 1
    pt_path = os.path.join(base_path, 'PT')
    try:
        for filename in os.listdir(pt_path):
            if filename.endswith('.csv'):
                filepath = os.path.join(pt_path, filename)
                df = pd.read_csv(filepath)
                data_list.append(df)
                labels_list.append(1) # Étiquette 1 pour PT
    except FileNotFoundError:
        print(f"Erreur: Le chemin PT n'a pas été trouvé: {pt_path}")
        return np.array([]), np.array([]), []
    except Exception as e:
        print(f"Une erreur s'est produite lors du chargement des données PT : {e}")
        return np.array([]), np.array([]), []

    # Vérifiez si des données ont été chargées
    if not data_list:
        print("Aucun fichier CSV trouvé ou les chemins d'accès sont incorrects.")
        return np.array([]), np.array([]), []

    # --- Option 1 : Chaque ligne est un exemple indépendant ---
    # Concaténation des DataFrames
    X_full = pd.concat(data_list, ignore_index=True)
    
    # Création du vecteur d'étiquettes
    y_full = np.concatenate([[label] * len(data) for data, label in zip(data_list, labels_list)])

    # 4. Séparer Features (X) et Labels (y) et nettoyer les colonnes
    columns_to_drop = ['file', 'start', 'end']
    
    # Assurez-vous que les colonnes existent avant de les supprimer
    cols_to_drop_present = [col for col in columns_to_drop if col in X_full.columns]
    X = X_full.drop(cols_to_drop_present, axis=1)

    # Sauvegarde des noms de features
    feature_names = X.columns.tolist()
    
    # Conversion en numpy array
    X = X.to_numpy()
    
    return X, y_full, feature_names

# ----------------------------------------------------------------------------------

# --- 2. FONCTION DE CLASSIFICATION ET ÉVALUATION ---
def test_k_folds_android():
    # 1. Chargement des données 
    X, y, feature_names = load_and_preprocess_data()
    
    # Vérification du chargement
    if X.size == 0 or y.size == 0:
        print("\nArrêt du script : Données de chargement non valides.")
        return

    # 2. Initialisation du classifieur
    # SVC avec un noyau linéaire est un classifieur linéaire.
    clf = SVC(kernel='linear', C=1, random_state=42)
    
    # 3. Validation Croisée (cv=5)
    # return_estimator=True est essentiel pour récupérer les modèles entraînés
    cv_results = cross_validate(
        clf, X, y, 
        cv=5, 
        scoring='accuracy', 
        return_estimator=True
    )
    
    # 4. Évaluation
    mean_accuracy = cv_results["test_score"].mean()
    print("\n--- ÉVALUATION PAR VALIDATION CROISÉE ---")
    print(f"Précision moyenne (5-Fold CV): {mean_accuracy:.4f}")
    
    # ----------------------------------------------------------------------------------
    # 5. Extraction et Analyse des Coefficients (MIS À JOUR)
    # ----------------------------------------------------------------------------------
    
    print("\n--- ANALYSE DES COEFFICIENTS DU CLASSIFIEUR PAR PLI ---")
    
    # Récupérer les 5 classifieurs entraînés
    estimators = cv_results["estimator"]
    
    # Liste pour stocker les DataFrames de coefficients par pli
    fold_coeffs_dfs = []
    all_coeffs = []

    # Itérer sur chaque estimateur entraîné
    for i, estimator in enumerate(estimators):
        fold_coeffs = estimator.coef_.flatten()
        all_coeffs.append(fold_coeffs)
        
        # Créer un DataFrame pour ce pli
        fold_df = pd.DataFrame({
            'Feature': feature_names,
            f'Fold_{i+1}_Coefficient': fold_coeffs 
        })
        fold_coeffs_dfs.append(fold_df.set_index('Feature'))
        
        # Affichage du top/bottom 3 pour le pli actuel
        print(f"\nCoefficients du Pli {i+1} (Top/Bottom 3) :")
        print(fold_df.sort_values(by=f'Fold_{i+1}_Coefficient', ascending=False).head(3))
        print(fold_df.sort_values(by=f'Fold_{i+1}_Coefficient', ascending=False).tail(3))
        

    # Concaténer tous les résultats des plis dans un seul DataFrame
    # Note: L'utilisation de pd.concat avec axis=1 nécessite que l'index (Feature) soit défini
    all_folds_combined_df = pd.concat(fold_coeffs_dfs, axis=1).reset_index()
    
    # Calculer la moyenne des coefficients sur les 5 folds
    mean_coeffs = np.mean(all_coeffs, axis=0)
    
    # Ajouter la moyenne au DataFrame combiné pour une vue complète
    all_folds_combined_df['Mean_Coefficient'] = mean_coeffs

    # Créer un DataFrame pour afficher le Top 5 de la MOYENNE (comme dans votre sortie originale)
    mean_df = pd.DataFrame({
        'Feature': feature_names,
        'Mean_Coefficient': mean_coeffs
    }).sort_values(by='Mean_Coefficient', ascending=False)
    
    # Affichage des résultats basés sur la moyenne
    print("\n" + "="*50)
    print("--- RÉSULTATS AGRÉGÉS (MOYENNE) ---")
    print(mean_df.head(5).to_string(index=False))
    
    print("\nTop 5 des features avec les coefficients négatifs les plus faibles (Favorise la classe 0 - HC):")
    print(mean_df.tail(5).to_string(index=False))
    print("="*50)
    
    print("\n--- APERÇU DU TABLEAU COMPLET (Coefficients par pli et Moyenne) ---")
    # Afficher les 10 premières lignes du DataFrame complet
    cols_order = ['Feature', 'Mean_Coefficient'] + [col for col in all_folds_combined_df.columns if col.startswith('Fold_')]
    print(all_folds_combined_df[cols_order].head(10).to_string(index=False))


if __name__ == '__main__':
    # Lancez la fonction d'évaluation
    test_k_folds_android()

**Task n°3. Plot the average relative contribution (%) of each feature with standard-deviation (e.g. using a box plot).**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

try:
    _ = all_folds_combined_df 
    print("Utilisation du DataFrame 'all_folds_combined_df' existant.")
except NameError:
    print("Le DataFrame 'all_folds_combined_df' n'est pas défini. Utilisation de données simulées pour le graphique.")

# Les colonnes de coefficients pour le calcul (identifiez-les dynamiquement si possible)
# On suppose que les colonnes de coefficients commencent par 'Fold_' et se terminent par '_Coefficient'
coeff_cols = [col for col in all_folds_combined_df.columns if col.startswith('Fold_') and col.endswith('_Coefficient')]

# S'assurer que 'Feature' est la première colonne pour faciliter le traitement
if 'Feature' in all_folds_combined_df.columns:
    df_plot_data = all_folds_combined_df[['Feature'] + coeff_cols].copy()
else:
    # Si 'Feature' est l'index, le réinitialiser
    df_plot_data = all_folds_combined_df[coeff_cols].copy()
    df_plot_data = df_plot_data.reset_index().rename(columns={'index': 'Feature'})


# Calculer la contribution relative (%) pour chaque pli
df_rel_contrib = df_plot_data[['Feature']].copy()

for col in coeff_cols:
    # Calculer la valeur absolue du coefficient pour la colonne actuelle
    abs_coeffs = df_plot_data[col].abs()
    
    # Calculer la somme totale des contributions absolues pour ce pli.
    # Ajouter une petite constante pour éviter la division par zéro si la somme est 0 (très rare)
    sum_abs_coeffs = abs_coeffs.sum() + 1e-9 # Ajout d'une petite valeur
    
    # Calculer la contribution relative en pourcentage
    df_rel_contrib[f'Relative_Contribution_{col.split("_")[1]}'] = (abs_coeffs / sum_abs_coeffs) * 100

# Mettre les données au format long pour le box plot (seaborn)
# On garde 'Feature' et on met les colonnes de contribution en lignes
df_long = df_rel_contrib.melt(
    id_vars='Feature', 
    value_vars=[col for col in df_rel_contrib.columns if col.startswith('Relative_Contribution')], 
    var_name='Fold', 
    value_name='Contribution (%)'
)

# Calculer la moyenne de la contribution pour trier les features dans le graphique
mean_contributions = df_long.groupby('Feature')['Contribution (%)'].mean().sort_values(ascending=False)
sorted_features = mean_contributions.index

# --- Configuration et Génération du Box Plot ---
plt.figure(figsize=(12, max(6, len(sorted_features) * 0.4))) # Taille dynamique du graphique
sns.boxplot(
    x='Contribution (%)', 
    y='Feature', 
    data=df_long, 
    order=sorted_features, 
    palette="viridis" # Ou une autre palette de couleurs comme "coolwarm", "plasma"
)
plt.title('Contribution Relative Moyenne des Caractéristiques (SVM Linéaire) avec Écart-type', fontsize=14)
plt.xlabel('Contribution Relative (%)', fontsize=12)
plt.ylabel('Caractéristique', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6) # Ajoute une grille horizontale
plt.tight_layout() # Ajuste automatiquement les paramètres de la figure pour les marges
plt.show() # Affiche le graphique

# Optionnel : Enregistrer le graphique dans un fichier
# plt.savefig('relative_contribution_boxplot.png', dpi=300) 
# print("Le graphique a été enregistré sous 'relative_contribution_boxplot.png'")

**Q1. What are the most important features?**

**Q2. Write a paragraph describing the impact of depression on read speech according to the classifier.**

### 1.2 Spontaneous speech
**Task n°4. Do the same for spontaneous speech.**

In [ ]:
# --- 1. FONCTION DE CHARGEMENT DES DONNÉES (CORRIGÉE) ---
def load_and_preprocess_data(base_path="/Users/klockjoely/Downloads/feautres_android_means/spontaneous_means"):
    """
    Charge et prépare les données du sous-corpus 'spontaneous' 
    à partir du fichier CSV agrégé en utilisant la colonne 'directory' pour les étiquettes.
    """
    
    # NOTE: Assurez-vous que le fichier 'directory_means.csv' est dans le dossier spécifié par base_path
    csv_file_path = os.path.join(base_path, '/Users/klockjoely/Downloads/directory_means.csv')
    
    try:
        # Tente de lire le fichier
        X_full = pd.read_csv(csv_file_path)
    except FileNotFoundError:
        # Si le fichier n'est pas trouvé
        print(f"Erreur: Le fichier {csv_file_path} n'a pas été trouvé. Vérifiez le chemin.")
        return np.array([]), np.array([]), []
    except Exception as e:
        print(f"Erreur de lecture du fichier CSV : {e}")
        return np.array([]), np.array([]), []

    # 1. CRÉATION DES ÉTIQUETTES à partir de la colonne 'directory'
    # 0 = Control (HC), 1 = Patient (PT)
    try:
        X_full['label'] = X_full['directory'].str[3].apply(lambda x: 0 if x == 'C' else 1).astype(int)
    except Exception as e:
        print(f"Erreur lors de l'extraction des étiquettes à partir de la colonne 'directory' : {e}")
        return np.array([]), np.array([]), []

    # 2. Séparer les étiquettes
    y_full = X_full['label'].to_numpy()

    # 3. Nettoyer les colonnes non-features
    columns_to_drop = ['directory', 'label']
    
    # Vérification et suppression des colonnes
    cols_to_drop_present = [col for col in columns_to_drop if col in X_full.columns]
    X = X_full.drop(cols_to_drop_present, axis=1)

    # Sauvegarde des noms de features et conversion
    feature_names = X.columns.tolist()
    X = X.to_numpy()
    
    return X, y_full, feature_names

# ----------------------------------------------------------------------------------

# --- 2. FONCTION DE CLASSIFICATION ET ÉVALUATION ---
def test_k_folds_android(plot=True):
    # 1. Chargement des données 
    X, y, feature_names = load_and_preprocess_data()
    
    if X.size == 0 or y.size == 0:
        print("\nArrêt du script : Données de chargement non valides.")
        return

    # 2. Initialisation du classifieur
    clf = SVC(kernel='linear', C=1, random_state=42)
    
    # 3. Validation Croisée (cv=5)
    cv_results = cross_validate(
        clf, X, y, 
        cv=5, 
        scoring='accuracy', 
        return_estimator=True
    )
    
    # 4. Évaluation
    mean_accuracy = cv_results["test_score"].mean()
    print("\n--- ÉVALUATION PAR VALIDATION CROISÉE (SPONTANEOUS) ---")
    print(f"Précision moyenne (5-Fold CV): {mean_accuracy:.4f}")
    
    # 5. Extraction des Coefficients (OUI, C'EST LE BLOC DE CODE QUE VOUS VOULEZ GARDER POUR L'ANALYSE)
    print("\n--- ANALYSE DES COEFFICIENTS DU CLASSIFIEUR PAR PLI ---")
    
    estimators = cv_results["estimator"]
    fold_coeffs_dfs = []
    all_coeffs = []

    for i, estimator in enumerate(estimators):
        fold_coeffs = estimator.coef_.flatten()
        all_coeffs.append(fold_coeffs)
        fold_df = pd.DataFrame({'Feature': feature_names, f'Fold_{i+1}_Coefficient': fold_coeffs })
        fold_coeffs_dfs.append(fold_df.set_index('Feature'))
        print(f"\nCoefficients du Pli {i+1} (Top/Bottom 3) :")
        print(fold_df.sort_values(by=f'Fold_{i+1}_Coefficient', ascending=False).head(3).to_string(index=False))
        print(fold_df.sort_values(by=f'Fold_{i+1}_Coefficient', ascending=False).tail(3).to_string(index=False))
        
    all_folds_combined_df = pd.concat(fold_coeffs_dfs, axis=1).reset_index()
    mean_coeffs = np.mean(all_coeffs, axis=0)
    all_folds_combined_df['Mean_Coefficient'] = mean_coeffs
    mean_df = pd.DataFrame({'Feature': feature_names, 'Mean_Coefficient': mean_coeffs}).sort_values(by='Mean_Coefficient', ascending=False)
    
    print("\n" + "="*50)
    print("--- RÉSULTATS AGRÉGÉS (MOYENNE) ---")
    print("\nTop 5 des features positives (Favorise la classe 1 - PT):")
    print(mean_df.head(5).to_string(index=False))
    print("\nTop 5 des features négatives (Favorise la classe 0 - HC):")
    print(mean_df.tail(5).to_string(index=False))
    print("="*50)
    
    # ----------------------------------------------------------------------------------
    # 6. Tracé du Box Plot de la Contribution Relative
    # ----------------------------------------------------------------------------------
    if plot:
        plot_relative_contribution(all_folds_combined_df)
        
    return all_folds_combined_df 

# --- 3. FONCTION DE TRACÉ DU BOX PLOT ---
def plot_relative_contribution(df_combined):
    """Génère le box plot de la contribution relative des coefficients par pli."""
    
    # Les colonnes de coefficients (Fold)
    coeff_cols = [col for col in df_combined.columns if col.startswith('Fold_') and col.endswith('_Coefficient')]

    df_plot_data = df_combined[['Feature'] + coeff_cols].copy()
    df_rel_contrib = df_plot_data[['Feature']].copy()

    for col in coeff_cols:
        abs_coeffs = df_plot_data[col].abs()
        sum_abs_coeffs = abs_coeffs.sum() + 1e-9 
        df_rel_contrib[f'Relative_Contribution_{col.split("_")[1]}'] = (abs_coeffs / sum_abs_coeffs) * 100

    df_long = df_rel_contrib.melt(
        id_vars='Feature', 
        value_vars=[col for col in df_rel_contrib.columns if col.startswith('Relative_Contribution')], 
        var_name='Fold', 
        value_name='Contribution (%)'
    )

    mean_contributions = df_long.groupby('Feature')['Contribution (%)'].mean().sort_values(ascending=False)
    sorted_features = mean_contributions.index

    # --- Génération du Box Plot ---
    plt.figure(figsize=(12, max(6, len(sorted_features) * 0.4)))
    sns.boxplot(
        x='Contribution (%)', 
        y='Feature', 
        data=df_long, 
        order=sorted_features, 
        palette="coolwarm"
    )
    plt.title('Contribution Relative Moyenne des Caractéristiques (SPONTANEOUS)', fontsize=14)
    plt.xlabel('Contribution Relative (%)', fontsize=12)
    plt.ylabel('Caractéristique', fontsize=12)
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    
    # LIGNE À RETIRER/COMMENTER POUR ÉVITER L'OSError :
    # plt.savefig('relative_contribution_spontaneous_boxplot.png') 
    
    plt.show()

if __name__ == '__main__':
    # Lancez l'analyse pour le sous-corpus spontané
    test_k_folds_android()

**Q3. Are the features that are important for reading the same as those for spontaneous speech?**